# Download texts from Project Gutenberg 

In [123]:
# Network requests
import requests

# Regex text matching
import re

# Parsing HTML
from bs4 import BeautifulSoup

# File system management
import os

# Timer
import timeit

In [120]:
page_index = 0 # Starts from 0
base_url = f'http://www.gutenberg.org'
book_page_extension = f'/ebooks/search/?sort_order=downloads&start_index={1+25*page_index}'
index = requests.get(base_url+book_page_extension).text
soup_index = BeautifulSoup(index, 'html.parser')

# Find the links that are dates of dumps
links = [(a['href'], a.contents[3].text.split("\n")[1].strip()) for a in soup_index.find_all('a') if 
         a.has_attr('href') and re.match(r"\/ebooks\/[0-9]+", a['href'], re.DOTALL)]
links[:5]

[('/ebooks/1342', 'Pride and Prejudice'),
 ('/ebooks/84', 'Frankenstein; Or, The Modern Prometheus'),
 ('/ebooks/1080', 'A Modest Proposal'),
 ('/ebooks/2542', 'Et dukkehjem. English'),
 ('/ebooks/25525', 'The Works of Edgar Allan Poe, The Raven Edition'),
 ('/ebooks/514', 'Little Women'),
 ('/ebooks/1635', 'Ion'),
 ('/ebooks/11', "Alice's Adventures in Wonderland"),
 ('/ebooks/844',
  'The Importance of Being Earnest: A Trivial Comedy for Serious People'),
 ('/ebooks/98', 'A Tale of Two Cities'),
 ('/ebooks/215', 'The Call of the Wild'),
 ('/ebooks/2701', 'Moby Dick; Or, The Whale'),
 ('/ebooks/76', 'Adventures of Huckleberry Finn'),
 ('/ebooks/1952', 'The Yellow Wallpaper'),
 ('/ebooks/16328', 'Beowulf: An Anglo-Saxon Epic Poem'),
 ('/ebooks/408', 'The Souls of Black Folk'),
 ('/ebooks/205', 'Walden, and On The Duty Of Civil Disobedience'),
 ('/ebooks/5200', 'Metamorphosis'),
 ('/ebooks/160', 'The Awakening, and Selected Short Stories'),
 ('/ebooks/1661', 'The Adventures of Sherlock 

In [113]:
def download(link):
    link_html = requests.get(base_url+link).text
    soup_link = BeautifulSoup(link_html, 'html.parser')
    
    text_links = [a['href'] for a in soup_link.find_all('a') if a.has_attr('href') and re.match(r"\/files\/[0-9]+\/([0-9a-zA-Z\.-])+txt", a['href'])]

    book_text_link = text_links[0]
    book_text = requests.get(base_url+book_text_link).text
    return book_text

def save(text, filename, folder='books'):
    if not os.path.exists(folder):
        os.mkdir(folder)
    with open(folder+'/'+filename+'.txt', 'w', encoding='utf') as f:
        f.writelines(text)

In [124]:
folder = 'books'

start = timeit.time.time()
for link, book_name in links:
    try:
        text = download(link)
    except Exception:
        pass
    save(text, book_name, folder)

end = timeit.time.time()

print(f'Downloaded {len(links)} links in {end-start} seconds. All texts saved in the folder {folder}')

Downloaded 25 links in 32.42083144187927 seconds. All texts saved in the folder books
